- https://pypi.org/project/panel/#:~:text=Bind%20it%20to%20a%20Panel%20widget%20and%20lay,web%20server%20wrap%20it%20in%20a%20nice%20template

- https://panel.holoviz.org/how_to/notebook/notebook.html

- https://panel.holoviz.org/reference/layouts/Tabs.html

- https://panel.holoviz.org/how_to/layout/examples/dynamic_tabs.html

- https://learn.deeplearning.ai/courses/langchain-chat-with-your-data/lesson/7/chat

In [16]:
import panel as pn
import param

In [17]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/cs229_lectures/MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [18]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

NameError: name 'load_db' is not defined

In [19]:
import ipywidgets as widgets
from IPython.display import display

# Create widgets for panel components
button1 = widgets.Button(description="Button 1")
button2 = widgets.Button(description="Button 2")
output = widgets.Output()

# Define callback functions for buttons
def on_button1_clicked(b):
    with output:
        print("Button 1 clicked")

def on_button2_clicked(b):
    with output:
        print("Button 2 clicked")

# Bind callback functions to button events
button1.on_click(on_button1_clicked)
button2.on_click(on_button2_clicked)

# Arrange widgets in a layout
panel = widgets.VBox([button1, button2, output])

# Display the panel
display(panel)

# **CREATING A PANEL**

In [42]:
import param
import panel as pn
import pandas as pd
pn.extension()

In [55]:
class Interactions(param.Parameterized):
    model = param.ObjectSelector(objects=['Random Forest', 'GNN', 'AE embeddings', 'AE reconstruction'])
    
    def __init__(self, **params):
        super().__init__(**params)
        self.data = None
        self.panels = []
        
    def load_file(self):
        # if file_input.value is None:
        #     # return pn.pane.Markdown(f"No file loaded or no correct format.")
        # else:
            button_load.button_style="outline"
            button_load.button_style="solid"
            print('2')
            with open(file_input.value, 'rb') as f:
                # file_input.value = 
                print('3')
                return pn.pane.Markdown(f"Loaded File: {file_input}")


    # def call_load_db(self, count):
    #     if count == 0 or file_input.value is None:  # init or no file specified :
    #         return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
    #     else:
    #         file_input.save("temp.pdf")  # local copy
    #         self.loaded_file = file_input.filename
    #         button_load.button_style="outline"
    #         self.qa = load_db("temp.pdf", "stuff", 4)
    #         button_load.button_style="solid"
    #     self.clr_history()
    #     return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

            
    @param.depends('model', watch=True)
    def models(self):
        if self.model == "Random Forest":
            # call function random forest model
            pass
            
        elif self.model == 'GNN':
            # call function GNN model
            pass
            
        elif self.model == 'AE embeddings':
            # call function AE embeddings
            pass
        
        else:
            # call function AE reconstruction
            pass

class results(param.Parameterized):
    pass

class visualizations(param.Parameterized):
    pass

In [57]:
interactions = Interactions()

# For tab 1: create buttons to enter the text and upload the file, also a button to clear the history
file_input = pn.widgets.FileInput(accept='.pkl')
button_load = pn.widgets.Button(name="Load file", button_type='primary')
button_load.param.watch(interactions.load_file(), 'clicks')
# bound_button_load = pn.bind(interactions.load_file, button_load.param.clicks)

tab1 = pn.Column(
    pn.layout.Divider(),
    pn.Row(),
    pn.Row(file_input, button_load),
    pn.Row(),
    pn.layout.Divider(),
)


# Tab 2: Creating options for the user to choose from
model_widget = pn.widgets.Select(name="Models", options=["Random Forest", "GNN", "Autoencoders Embeddings", "Autoencoders Reconstruction"])
model_widget2 = pn.widgets.MultiSelect(name="Models", value=["GNN", "Autoencoders Embeddings"],
                                       options=["Random Forest", "GNN", "Autoencoders Embeddings", "Autoencoders Reconstruction"])
timestep_widget = pn.widgets.IntSlider(name="TimeStep", value=30, start=1, end=49)
button_get_predictions = pn.widgets.Button(name="Get Predictions", button_type='primary')

tab2= pn.Column(
    pn.layout.Divider(),
    pn.Row(),
    pn.Row(model_widget, model_widget2),
    pn.Row(),
    pn.Row(timestep_widget),
    pn.Row(),
    pn.Row(button_get_predictions),
    pn.Row(),
    pn.layout.Divider(),
)


# Tab 3: create visualizations of the original graph and the predicted graph
from bokeh.plotting import figure

plot = figure(width=300, height=300, name='Scatter')
plot.scatter([0, 1, 2, 3, 4, 5, 6], [0, 1, 2, 3, 2, 1, 0])

tab3= pn.Column(
    pn.layout.Divider(),
    pn.Row(),
    pn.Row(plot),
    pn.Row(),
    pn.layout.Divider(),
)


# Tab 4: Display results (nº of real licit, ilicit and unknown nodes vs nº of predicted (table/plot))
tab4=pn.Column(
    pn.layout.Divider(),
)


# Unificate all the characteristics into a panel
panel_graph = pn.Column(pn.Row(pn.pane.Markdown('# Visualize Graphs')),
                        pn.Tabs(('Upload File', tab1), ('Model and Time Step', tab2), ('Visualizations', tab3),('Results', tab4)))
panel_graph

2


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [50]:
pn.serve(panel_graph)

Launching server at http://localhost:55495


AssertionError: 